In [ ]:
# ***********************************************************************************
# Changelog
# *******

# Code written by Ousmane Diao 


#*************************************************************************************************
# ***********************************************************************************
# Imports
# *******
%matplotlib inline

from ipywidgets import interact, interactive
from IPython.display import clear_output, display, HTML
import warnings
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Poisson, NegativeBinomial, Gaussian
from statsmodels.genmod.families.links import identity, log, sqrt
import matplotlib.pyplot as plt
import math 
from scipy.special import factorial, gamma
from scipy import stats
from scipy.optimize import least_squares
from scipy import optimize

import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
import os
#os.chdir ('C:\\Users\\odiao\\Desktop\\Model paludisme')
os.chdir('C:\\Users\\odiao\\Dropbox\\Model paludisme')
df = pd.read_csv('Dakar.csv', header=0, infer_datetime_format=False, parse_dates=[0])
bd = pd.read_csv('Data_by_district.csv', header=0, infer_datetime_format=False, parse_dates=[0])

#***************************************************************************************************
#Defined the avaerage temperature
AVT_Dakar=(df.Tempmin_Dakar + df.Tempmax_Dakar)/2
AVT_Fatick=(df.Tempmin_Fatick + df.Tempmax_Fatick)/2
AVT_Kedougou=(df.Tempmin_Kedougou + df.Tempmax_Kedougou)/2


warnings.filterwarnings("ignore") 

In [ ]:
# ***********************************************************************************
# Define several functions: the mean function, the log-likelihood function, the mass functions, the least-squares optimization functions of the saturation and the link function (the link can be exponential or identity or sqrt). 
# *******  
  #Define the mean function: for example link=identity
def mu(X, beta):
    return X @ beta  #Defined with link=id
    #Define the log-likelihood function of Poisson distribution
def ll_P(beta, X, y):
    return -np.sum( y * np.log(mu(X, beta)) - mu(X, beta)) #- np.log(gamma(y+1)
    #Define the log-likelihood function of negative binomial distribution
def ll_nb(beta, X, y):
    L_y=np.full(len(y), np.nan)
    for i in np.arange(0,len(y)):
        L_y[i] = np.sum(np.log(np.arange(0, y[i]) + 1/beta[0]))
    return -np.sum(y * np.log(beta[0]*mu(X, beta[1:])/(1+beta[0]*mu(X, beta[1:]))) - 1/beta[0]*np.log(1+beta[0]*mu(X, beta[1:])) +L_y) #- np.log(gamma(y+1)) +np.log(gamma(y+1/beta[0])) - np.log(gamma(1/beta[0]))

#*************************************************************************************
#******************Define the forecasting function*******************************************************************
def forecast_functions(sw_protocols, sw_link, t_c, t_e, lag, h, coeff, X):
    forecasts=np.full(t_e, np.nan)  # set storage
    if sw_protocols=='P1':
        for t in np.arange(t_c,t_e):
            if sw_link==identity:
                forecasts[t] = coeff[0]*X[t-max(lag[0],h),0] + coeff[1]*X[t-max(lag[1],h),1] + coeff[2]*X[t-max(lag[2],h),2] + coeff[3]*X[t-max(lag[3],h),3] + coeff[4]*X[t-h,4]
            if sw_link==log:
                forecasts[t] = np.exp(coeff[0]*X[t-max(lag[0],h),0] + coeff[1]*X[t-max(lag[1],h),1] + coeff[2]*X[t-max(lag[2],h),2] + coeff[3]*X[t-max(lag[3],h),3] + coeff[4]*X[t-h,4])
            if sw_link==sqrt:
                forecasts[t] = (coeff[0]*X[t-max(lag[0],h),0] + coeff[1]*X[t-max(lag[1],h),1] + coeff[2]*X[t-max(lag[2],h),2] + coeff[3]*X[t-max(lag[3],h),3] + coeff[4]*X[t-h,4])**2
    if sw_protocols=='P2':
        for t in np.arange(t_c,t_e):
            if sw_link==identity:                                                                                                                             
                forecasts[t]=coeff[0]*X[t,0] + coeff[1]*X[t,1] + coeff[2]*X[t,2] + coeff[3]*X[t-max(lag[3],h),3] + coeff[4]*X[t-h,4]                                                                                                                                   
            if sw_link==log:
                forecasts[t]=np.exp(coeff[0]*X[t,0] + coeff[1]*X[t,1] + coeff[2]*X[t,2] + coeff[3]*X[t-max(lag[3],h),3] + coeff[4]*X[t-h,4])
            if sw_link==sqrt:
                forecasts[t] = (coeff[0]*X[t,0] + coeff[1]*X[t,1] + coeff[2]*X[t,2] + coeff[3]*X[t-max(lag[3],h),3] + coeff[4]*X[t-h,4])**2  
    return forecasts[t_c:t_e]

# Define function that gathers statistics:
def accuracy_measures(sw_regions,sw_model,sw_link,sw_show_results,mu_fitted, mu_forecasted, y_train, y_test, t_i, t_c, t_e, y_o,min_fitted):
    if sw_show_results=='all_links':
        stats_all=np.full((3,9), np.nan)
    if sw_show_results=='each_link':
        stats_all=np.full(9, np.nan)
    R2_train=np.corrcoef(y_train, mu_fitted)[0][1]**2
    R2_test=np.corrcoef(y_test, mu_forecasted)[0][1]**2
    RMSE_train = np.linalg.norm(mu_fitted - y_train) / math.sqrt(t_c-t_i)
    RMSE_test = np.linalg.norm(mu_forecasted - y_test) / math.sqrt(t_e-t_c)
    MAE_train = np.mean(np.abs(y_train - mu_fitted))
    MAE_test =  np.mean(np.abs(y_test - mu_forecasted))
    MASE_train = MAE_train / np.mean(np.abs(y_o[(t_i+1):t_c] - y_o[t_i:(t_c-1)]))
    MASE_test = MAE_test / np.mean(np.abs(y_o[t_c+1:t_e] - y_o[t_c:t_e-1]))  # MAE divided by MAE of the prescient naive one-step-ahead predictor (that predicts total_in[t] by total_in[t-1]). Since the decrease is slow, this can be interpreted as the MAE divided by the noise level. If it gets below 1, then the fit is visually excellent. This measure is strongly inspired from Hyndman & Koehler 2006 (https://doi.org/10.1016/j.ijforecast.2006.03.001).
    MARE_train = np.mean(np.abs((mu_fitted -  y_train) /  y_train))
    MARE_test = np.mean(np.abs((mu_forecasted - y_test) / y_test))
    Min = min_fitted
    stats_all=[RMSE_train,RMSE_test,MASE_train,MASE_test,MARE_train,MARE_test,R2_train,R2_test,Min] 
    if sw_model==Gaussian:
        sw_model='Gausian' 
    if sw_model==Poisson:
        sw_model='Poisson'
    if sw_model==NegativeBinomial:
        sw_model='NB' 
    if sw_link==identity:
        sw_link='id'
    if sw_link==log:
        sw_link='log'
    if sw_link==sqrt:
        sw_link='sqrt'
    print(sw_regions,"&",sw_model,"&",sw_link, "&", round(stats_all[0],2),"/",round(stats_all[1],2), '&', round(stats_all[2],2) ,"/", round(stats_all[3],2),'&', round(stats_all[4],2) ,"/",  round(stats_all[5],2), '&', round(stats_all[6],2) ,"/",  round(stats_all[7],2), '&', round(stats_all[8],2),'\\\\')
    
    return 
#**************Define the Plots function***************************************
def figures(sw_protocols, y_o, t_i, t_c, t_e, lag, h, mu_fitted, mu_train_lower, mu_train_upper, mu_forecasted,params, X):
    #We show the plots
    fig=plt.figure(figsize=(14,7))
    plt.subplot(121)
    plt.plot(df.Date, y_o, color = "black" ,  label="Malaria incidence count", marker='o', markersize=8)
    plt.plot(df.Date[t_i:t_c], mu_fitted, color = "blue", label= "predicted data",marker='o', markersize=8)
    plt.plot(df.Date[t_i:t_c], mu_train_lower, "y--", label="data_lower", linewidth=2)
    plt.plot(df.Date[t_i:t_c], mu_train_upper, "g--", label="data_lower", linewidth=2)
    plt.plot(df.Date[t_c:t_e], mu_forecasted, color = "red", label="forecasted data", marker='o', markersize=8)
    plt.xlabel("Date [months]", fontsize=20)
    plt.title("A", fontsize=20) 
    plt.ylabel("Malaria incidence", fontsize=20)
    plt.legend(fontsize=15)
    plt.subplot(122)
    if sw_protocols=='P1':
        plt.plot(df.Date[t_i-max(lag[0],h):t_e-max(lag[0],h)], params[0]*X[t_i-max(lag[0],h):t_e-max(lag[0],h),0], color="green", label="rainfall")
        plt.plot(df.Date[t_i-max(lag[1],h):t_e-max(lag[1],h)], params[1]*X[t_i-max(lag[1],h):t_e-max(lag[1],h),1], color="red", label="avg_temperature")
        plt.plot(df.Date[t_i-max(lag[2],h):t_e-max(lag[2],h)], params[2]*X[t_i-max(lag[2],h):t_e-max(lag[2],h),2], color="cyan", label="humidity")
    if sw_protocols=='P2':
        plt.plot(df.Date[t_i:t_e], params[0]*X[t_i:t_e,0], color="green", label="rainfall")
        plt.plot(df.Date[t_i:t_e], params[1]*X[t_i:t_e,1], color="red", label="avg_temperature")
        plt.plot(df.Date[t_i:t_e], params[2]*X[t_i:t_e,2], color="cyan", label="humidity")
    plt.plot(df.Date[t_i-max(lag[3],h):t_e-max(lag[3],h)], params[3]*X[t_i-max(lag[3],h):t_e-max(lag[3],h),3], color="black", label="malaria cases in the past")
    plt.axhline(y=params[4], color="magenta", label="Intercept")                           
    plt.title("B", fontsize=20)
    plt.ylabel("β_iX_j", fontsize=20)
    plt.xlabel("Date [months]", fontsize=20)
    plt.legend(fontsize=15)
    #fig.savefig('figures_palu_rapport/MC_R_T_H_Poi_id_2_dk_2022_05_10.png')   # save the figure to file
    #plt.close(fig)
    return


In [ ]:
#***************** Regression coefficient exchange between the regions
#We train the GLM models on a specific region and we make the forecasts with these coeff in the others regions
#We make that for all the regions
#************************* Statistics *************************************
t_i=5; t_c=84; t_e=108; intercept=np.ones(t_e) ; regions = ['Dakar','Fatick','Kedougou']
y_o = np.c_[bd.iloc[:,7].values, bd.iloc[:,14].values, bd.iloc[:,17].values]
y_test = np.c_[bd.iloc[t_c:t_e,7].values, bd.iloc[t_c:t_e,14].values, bd.iloc[t_c:t_e,17].values]
#****************** Train and test sets
#********************* Dakar
X_dk = np.c_[df.Rainfall, AVT_Dakar, df.Humidity_D, bd.iloc[:,7], intercept]
X_train_dk = np.c_[X_dk[t_i-2:t_c-2,0], X_dk[t_i-2:t_c-2,1], X_dk[t_i-5:t_c-5,2], X_dk[t_i-1:t_c-1,3], X_dk[t_i:t_c,4]]
X_test_dk = np.c_[X_dk[t_c-2:t_e-2,0], X_dk[t_c-2:t_e-2,1], X_dk[t_c-5:t_e-5,2], X_dk[t_c-1:t_e-1,3], X_dk[t_c:t_e,4]]
#********************* Fatick
X_ft = np.c_[df.Rainfall_F, AVT_Fatick, df.Humidity_F, bd.iloc[:,14], intercept]
X_train_ft = np.c_[X_ft[t_i-3:t_c-3,0], X_ft[t_i-4:t_c-4,1], X_ft[t_i-3:t_c-3,2], X_ft[t_i-1:t_c-1,3], X_ft[t_i:t_c,4]]
X_test_ft = np.c_[X_ft[t_c-3:t_e-3,0], X_ft[t_c-4:t_e-4,1], X_ft[t_c-3:t_e-3,2], X_ft[t_c-1:t_e-1,3], X_ft[t_c:t_e,4]]
#********************** Kedouogu
X_kd = np.c_[df.Rainfall_K, AVT_Kedougou, df.Humidity_K, bd.iloc[:,17], intercept]
X_train_kd = np.c_[X_kd[t_i-2:t_c-2,0], X_kd[t_i-5:t_c-5,1], X_kd[t_i-2:t_c-2,2], X_kd[t_i-1:t_c-1,3], X_kd[t_i:t_c,4]]
X_test_kd = np.c_[X_kd[t_c-2:t_e-2,0], X_kd[t_c-5:t_e-5,1], X_kd[t_c-2:t_e-2,2], X_kd[t_c-1:t_e-1,3], X_kd[t_c:t_e,4]]
for sw_regions in regions:
    print("*****************************************************************************")
    print("************************", sw_regions, "*******************************") 
    if sw_regions=='Dakar': 
        #lag=[2,2,5,1] 
        y_train_dk = y_o[t_i:t_c,0]
        model = sm.GLM(y_train_dk, X_train_dk, family=Poisson(link=identity)).fit()
        print(model.summary())
        predictions_dk = model.mu
        forecasts_dk = X_test_dk@model.params
        forecasts_ft = X_test_ft@model.params
        forecasts_kd = X_test_kd@model.params
        
        ##*************** Plots
        fig=plt.figure(figsize = [12, 14])
        plt.subplot(311)
        plt.plot(df.Date, y_o[:,0], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_i:t_c], predictions_dk, color = "blue", label="Fit", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_dk, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        #plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Dakar: model trained in "+str(sw_regions),fontsize=20)
        #plt.show()
        plt.subplot(312)
        plt.plot(df.Date, y_o[:,1], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_ft, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        #plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Fatick: model trained in "+str(sw_regions),fontsize=20)

        plt.subplot(313)
        plt.plot(df.Date, y_o[:,2], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_kd, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Kedougou: model trained in "+str(sw_regions),fontsize=20)
        plt.show()
    if sw_regions=='Fatick': 
        #lag=[3,4,3,1]
        y_train_ft = y_o[t_i:t_c,1]
        model = sm.GLM(y_train_ft, X_train_ft, family=Poisson(link=identity)).fit()
        print(model.summary())
        predictions_ft = model.mu
        forecasts_dk = X_test_dk@model.params
        forecasts_ft = X_test_ft@model.params
        forecasts_kd = X_test_kd@model.params
        
        ##*************** Plots
        fig=plt.figure(figsize = [12, 14])
        plt.subplot(311)
        plt.plot(df.Date, y_o[:,1], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_i:t_c], predictions_ft, color = "blue", label="Fit", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_ft, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        #plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Fatick: model trained in "+str(sw_regions),fontsize=20)
        #plt.show()
        plt.subplot(312)
        plt.plot(df.Date, y_o[:,0], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_dk, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        #plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Dakar: model trained in "+str(sw_regions),fontsize=20)

        plt.subplot(313)
        plt.plot(df.Date, y_o[:,2], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_kd, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Kedougou: model trained in "+str(sw_regions),fontsize=20)
        plt.show()
        
    if sw_regions=='Kedougou': 
        #lag=[2,5,2,1]
        y_train_kd = y_o[t_i:t_c,2]
        model = sm.GLM(y_train_kd, X_train_kd, family=Poisson(link=identity)).fit()
        print(model.summary())
        predictions_kd = model.mu
        forecasts_dk = X_test_dk@model.params
        forecasts_ft = X_test_ft@model.params
        forecasts_kd = X_test_kd@model.params
    
        ##*************** Plots
        fig=plt.figure(figsize = [12, 14])
        plt.subplot(311)
        plt.plot(df.Date, y_o[:,2], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_i:t_c], predictions_kd, color = "blue", label="Fit", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_kd, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        #plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Kedougou: model trained in "+str(sw_regions),fontsize=20)
        #plt.show()
        plt.subplot(312)
        plt.plot(df.Date, y_o[:,0], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_dk, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        #plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Dakar: model trained in "+str(sw_regions),fontsize=20)

        plt.subplot(313)
        plt.plot(df.Date, y_o[:,1], color = "black" ,  label="Data", marker='o', markersize=8)
        plt.plot(df.Date[t_c:t_e], forecasts_ft, color = "red", label="Forecast", marker='o', markersize=8)
        plt.legend(loc='upper left', fontsize=20)
        plt.xlabel("Months",fontsize=20)
        plt.yticks(fontsize=20)
        plt.xticks(fontsize=20)
        plt.ylabel("Malaria incidence",fontsize=20)
        plt.title("Fatick: model trained in "+str(sw_regions),fontsize=20)
        plt.show()
    
    forecasts_region = np.c_[forecasts_dk, forecasts_ft, forecasts_kd]
    RMSE_test_0 = np.linalg.norm(forecasts_region[:,0] - y_test[:,0]) / math.sqrt(t_e-t_c)
    MAE_test_0 = np.mean(np.abs(forecasts_region[:,0] - y_test[:,0]))
    MASE_test_0 = MAE_test_0 / np.mean(np.abs(y_o[(t_c+1):t_e,0] - y_o[t_c:(t_e-1),0]))
    
    RMSE_test_1 = np.linalg.norm(forecasts_region[:,1] - y_test[:,1]) / math.sqrt(t_e-t_c)
    MAE_test_1 = np.mean(np.abs(forecasts_region[:,1] - y_test[:,1]))
    MASE_test_1 = MAE_test_1 / np.mean(np.abs(y_o[(t_c+1):t_e,1] - y_o[t_c:(t_e-1),1]))
    
    RMSE_test_2 = np.linalg.norm(forecasts_region[:,2] - y_test[:,2]) / math.sqrt(t_e-t_c)
    MAE_test_2 = np.mean(np.abs(forecasts_region[:,2] - y_test[:,2]))
    MASE_test_2 = MAE_test_2 / np.mean(np.abs(y_o[(t_c+1):t_e,2] - y_o[t_c:(t_e-1),2]))
    
    print("Model trained in: "+str(sw_regions)+ "RMSE_test's: "+str(np.c_[RMSE_test_0, RMSE_test_1, RMSE_test_2])+ "MASE_test's: "+str(np.c_[MASE_test_0, MASE_test_1, MASE_test_2]))  